In [ ]:
pip install geopy

In [13]:
import requests
from math import radians, sin, cos, sqrt, atan2
import pandas as pd
import time
import numpy as np

def get_coordinates(postal_code, country):
    """
    Ottiene le coordinate geografiche di un codice postale per IT o US
    """
    if country not in ['IT', 'US']:
        raise ValueError(f"Paese non supportato: {country}")
    
    # Costruisce la query appropriata per il paese
    if country == 'IT':
        query = f"{postal_code}, Italy"
    else:
        query = f"{postal_code}, United States"
    
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": query,
        "format": "json",
        "limit": 1,
        "countrycodes": country.lower()
    }
    headers = {
        "User-Agent": "DistanceCalculator/1.0"
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        if not data:
            raise ValueError(f"Codice postale non trovato: {postal_code} in {country}")
        
        # Rispetta il rate limiting di Nominatim
        time.sleep(1)
        
        return {
            "lat": float(data[0]["lat"]),
            "lon": float(data[0]["lon"])
        }
    except requests.exceptions.RequestException as e:
        raise Exception(f"Errore nella richiesta API: {str(e)}")

def calculate_distance(coord1, coord2):
    """
    Calcola la distanza in kilometri tra due punti usando la formula di Haversine
    """
    R = 6371  # Raggio della Terra in km
    
    lat1, lon1 = radians(coord1["lat"]), radians(coord1["lon"])
    lat2, lon2 = radians(coord2["lat"]), radians(coord2["lon"])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance

def process_csv(file_path):
    """
    Legge il CSV e aggiorna le distanze mancanti nella colonna DECLARED_DISTANCE_KM
    per spedizioni IT-IT, US-US, IT-US e US-IT
    """
    try:
        # Legge il CSV
        df = pd.read_csv(file_path)
        
        # Se non esiste la colonna DECLARED_DISTANCE_KM, la crea
        if 'DECLARED_DISTANCE_KM' not in df.columns:
            df['DECLARED_DISTANCE_KM'] = np.nan
        
        # Filtra le spedizioni IT e US
        mask = (
            ((df['DEPARTURE_COUNTRY'].isin(['IT', 'US'])) & 
             (df['ARRIVAL_COUNTRY'].isin(['IT', 'US'])))
        )
        valid_rows = df[mask].index
        
        if len(valid_rows) == 0:
            raise ValueError("Nessuna spedizione IT/US trovata nel CSV")
        
        # Statistiche iniziali
        stats = {
            'IT-IT': len(df[(df['DEPARTURE_COUNTRY'] == 'IT') & (df['ARRIVAL_COUNTRY'] == 'IT')]),
            'US-US': len(df[(df['DEPARTURE_COUNTRY'] == 'US') & (df['ARRIVAL_COUNTRY'] == 'US')]),
            'IT-US': len(df[(df['DEPARTURE_COUNTRY'] == 'IT') & (df['ARRIVAL_COUNTRY'] == 'US')]),
            'US-IT': len(df[(df['DEPARTURE_COUNTRY'] == 'US') & (df['ARRIVAL_COUNTRY'] == 'IT')])
        }
        
        print("Spedizioni trovate:")
        for route, count in stats.items():
            print(f"{route}: {count}")
        
        # Conta quante distanze devono essere calcolate
        missing_distances = df.loc[valid_rows, 'DECLARED_DISTANCE_KM'].isna().sum()
        print(f"\nDistanze da calcolare: {missing_distances}")
        
        # Elabora le righe con distanza mancante
        rows_processed = 0
        for idx in valid_rows:
            # Salta se la distanza è già presente
            if pd.notna(df.at[idx, 'DECLARED_DISTANCE_KM']):
                continue
                
            try:
                departure_code = str(df.at[idx, 'DEPARTURE_ZIPCODE'])
                arrival_code = str(df.at[idx, 'ARRIVAL_ZIPCODE'])
                departure_country = df.at[idx, 'DEPARTURE_COUNTRY']
                arrival_country = df.at[idx, 'ARRIVAL_COUNTRY']
                
                # Ottiene le coordinate
                coord1 = get_coordinates(departure_code, departure_country)
                coord2 = get_coordinates(arrival_code, arrival_country)
                
                # Calcola la distanza
                distance = calculate_distance(coord1, coord2)
                df.at[idx, 'DECLARED_DISTANCE_KM'] = round(distance, 2)
                
                rows_processed += 1
                print(f"Elaborata riga {idx + 1}: {departure_code}({departure_country}) -> "
                      f"{arrival_code}({arrival_country}) = {distance:.2f} km ({rows_processed}/{missing_distances})")
                
                # Salva periodicamente ogni 10 righe elaborate
                if rows_processed % 10 == 0:
                    df.to_csv(file_path, index=False)
                    print(f"Salvaggio intermedio effettuato dopo {rows_processed} righe")
                
            except Exception as e:
                print(f"Errore nella riga {idx + 1}: {str(e)}")
                continue
        
        # Salvataggio finale
        df.to_csv(file_path, index=False)
        print(f"\nFile aggiornato salvato in: {file_path}")
        
        # Mostra statistiche finali per ogni tipo di percorso
        print("\nStatistiche delle distanze per tipo di percorso:")
        for dep_country in ['IT', 'US']:
            for arr_country in ['IT', 'US']:
                route_mask = (df['DEPARTURE_COUNTRY'] == dep_country) & (df['ARRIVAL_COUNTRY'] == arr_country)
                route_distances = df[route_mask]['DECLARED_DISTANCE_KM'].dropna()
                
                if not route_distances.empty:
                    print(f"\n{dep_country}-{arr_country}:")
                    print(f"Numero spedizioni: {len(route_distances)}")
                    print(f"Distanza media: {route_distances.mean():.2f} km")
                    print(f"Distanza massima: {route_distances.max():.2f} km")
                    print(f"Distanza minima: {route_distances.min():.2f} km")
        
        return df
        
    except Exception as e:
        raise Exception(f"Errore nell'elaborazione del CSV: {str(e)}")

# Esempio di utilizzo
if __name__ == "__main__":
    try:
        input_file = input("Inserisci il percorso del file CSV da elaborare: ")
        results = process_csv(input_file)
        
    except Exception as e:
        print(f"Errore: {str(e)}")

Inserisci il percorso del file CSV da elaborare:  ./delivery_data.csv


Spedizioni trovate:
IT-IT: 11755
US-US: 2225
IT-US: 574
US-IT: 0

Distanze da calcolare: 2800
Elaborata riga 7: 62010.0(IT) -> 94551(US) = 9924.91 km (1/2800)
Elaborata riga 8: 62010.0(IT) -> 33154(US) = 8331.21 km (2/2800)
Elaborata riga 9: 62010.0(IT) -> 89109(US) = 9753.77 km (3/2800)
Elaborata riga 10: 62010.0(IT) -> 10022(US) = 6859.16 km (4/2800)
Elaborata riga 12: 62010.0(IT) -> 33154(US) = 8331.21 km (5/2800)
Errore nella riga 55: Errore nella richiesta API: 503 Server Error: Service Unavailable for url: https://nominatim.openstreetmap.org/search?q=95116.0%2C+United+States&format=json&limit=1&countrycodes=us
Elaborata riga 56: 33154.0(US) -> 10011(US) = 881.82 km (6/2800)
Elaborata riga 58: 7626.0(US) -> 7008(US) = 899.29 km (7/2800)
Elaborata riga 60: 10022.0(US) -> 92656(US) = 1079.07 km (8/2800)
Elaborata riga 61: 10022.0(US) -> 33140(US) = 3177.72 km (9/2800)
Elaborata riga 62: 10022.0(US) -> 34236(US) = 2884.79 km (10/2800)
Salvaggio intermedio effettuato dopo 10 righe
Ela